# ICD - Grupo 17: Ranqueamento de Edições da Final Mundial do ICPC

Alunos:

*   Ivanir Paulo Cardoso Ignacchitti
*   Luis Henrique Gomes Higino
*   Mateus Vitor Mota Vasconcelos


# Introdução
Neste trabalho será realizada uma exploração da base de dados de rankings da competição mundial do [ICPC](https://icpc.global/), a maratona mundial de programação. Aqui, buscamos responder às seguintes perguntas:
- Os países sede da competição tem alguma vantagem nos rankings?
- Qual o rendimento histórico dos times brasileiros em relação a times de outros países?
- Com que acurácia é possível prever um resultado futuro de acordo com o resultado histórico?


# Metodologia
Para este trabalho, foi utilizada a base de dados ICPC WF Ranking Results (1999 - Present) disponível no site [Kaggle](https://www.kaggle.com/datasets/justinianus/icpc-world-finals-ranking-since-1999/data). Nela, temos uma lista com as equipes universitárias que participaram da disputa entre os anos de 1999 e 2023. Para este trabalho, foram utilizados os dados até o ano de 2021.

Cada linha da base de dados corresponde a uma equipe que participou da disputa em alguma das edições. Entre os campos de informação, temos tanto informações sobre a equipe (ranking, universidade, país de origem, nomes dos participantes, medalhas e pontuação) quanto sobre a competição em questão (a data, o local, o país sede e a cidade).

Nós optamos por exibir as médias dos rankings em forma gráfica para comparar grupos distintos, como os times brasileiros e os não-brasileiros, ou times do país sede e de países estrangeiros.

Assim, nós observamos um comportamento esperado, e então quantificamos essas diferenças com o uso de intervalos de confiança, criados por meio de bootstrap, para descobrir o quão expressiva é a diferença em cada pergunta realizada.

Por fim, nós utilizamos de florestas aleatórias de decisão para predizer o resultado de países, e medimos sua acurácia com outra porção dos dados.

# Carregamento dos Dados

In [ ]:
!gdown 1JUBdJiq8HamFeeoMkiFKP4fdaeQ3S9dc

Downloading...
From: https://drive.google.com/uc?id=1JUBdJiq8HamFeeoMkiFKP4fdaeQ3S9dc
To: /content/icpc-full.csv
100% 462k/462k [00:00<00:00, 73.4MB/s]


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import scipy.stats as stats
import numpy as np

pd.options.plotting.backend = "plotly"
import plotly.io as pio
pio.templates.default = "plotly_dark"

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


## Pré-Processamento

Ao fazer uma análise manual dos dados e cruzar as informações com outras fontes, foi notado que times que se classificaram para a final mundial mas não compareceram ou tiveram sua participação desclassificada aparecem com a coluna "Rank" nula. Como essas instâncias não agregam muito na análise dos resultados, essas linhas vão ser descartadas.

In [ ]:
df = pd.read_csv('icpc-full.csv')
df = df.dropna(subset=['Rank'])
df.columns

Index(['Year', 'Date', 'Host', 'City', 'Venue', 'Rank', 'University',
       'Country', 'Team', 'Contestant 1', 'Contestant 2', 'Contestant 3',
       'Gold', 'Silver', 'Bronze', 'Honorable', 'Score', 'Total',
       'Score Percentage', 'Penalty', 'Prize'],
      dtype='object')

As principais colunas são o Rank, que indica a posição final do time e o conjunto de colunas que identifica o time e a competição (Year, Host, University e Team). As outras colunas trazem informações menos interessantes do ponto de vista da Analíse Exploratória e Predição/Classificação.

Abaixo seguem os últimos campeões mundiais como exemplos de instâncias da base de dados.

In [ ]:
df[df['Rank'] == 1].sort_values(by = 'Year', ascending=False).head(5)

,Year,Date,Host,City,Venue,Rank,University,Country,Team,Contestant 1,...,Contestant 3,Gold,Silver,Bronze,Honorable,Score,Total,Score Percentage,Penalty,Prize
2176,2021,2022-11-10,Bangladesh,Dhaka,International Convention City Bashundhara,1.0,Massachusetts Institute of Technology,United States,MIT ZEROONE,Jerry Mao,...,Xiao Mao,True,False,False,False,11.0,12,0.92,1339.0,"World Champion, North America Champion"
2059,2020,2021-10-05,Russia,Moscow,Moscow Manege,1.0,Nizhny Novgorod State University,Russia,Almost Retired,Aleksei Daniliuk,...,Valeria Ryabchikova,True,False,False,False,12.0,15,0.80,1714.0,"World Champion, Northern Eurasia Champion"
1924,2019,2019-04-04,Portugal,Porto,Alfândega Congress Centre,1.0,Moscow State University,Russia,Moscow SU: Red Panda,Grigory Reznikov,...,Vladislav Makeev,True,False,False,False,10.0,11,0.91,1531.0,"World Champion, Northern Eurasia Champion"
1784,2018,2018-04-19,China,Beijing,Peking University's Khoo Teck Puat Gymnasium,1.0,Moscow State University,Russia,Moscow SU Red Panda,Grigory Reznikov,...,Vladislav Makeev,True,False,False,False,9.0,11,0.82,1427.0,"World Champion, Northern Eurasia Champion"
1651,2017,2017-05-24,United States,"Rapid City, SD",Rushmore Plaza Civic Center,1.0,St. Petersburg ITMO University,Russia,SPb ITMO University 1,Ilya Zban,...,Vladimir Smykalov,True,False,False,False,10.0,12,0.83,1093.0,"World Champion, Europe Champion"


O rank em cada ano tem escalas diferentes devido ao número variado de participantes de cada edição. Vamos normalizar a coluna Rank em cada edição para um valor proporcional a 100 para tornar possível uma comparação direta entre edições diferentes.

In [ ]:
# Agrupa o DataFrame por ano e calcula o número total de participantes em cada ano
grouped = df.groupby('Year')['Rank'].max()

# Normaliza a coluna 'Rank' dividindo pelo número total de participantes em cada ano
for year, max_rank in grouped.items():
    df.loc[df['Year'] == year, 'Rank'] = df.loc[df['Year'] == year, 'Rank'] / max_rank * 100

# EDA

## Número de times participantes do país sede em cada edição

In [ ]:
grouped_df = df[['Year', 'Host', 'Team']].groupby(['Year', 'Host']).count()
flat_df = grouped_df.reset_index()
fig = go.Figure()
for host in flat_df['Host'].unique():
    host_df = flat_df[flat_df['Host'] == host]
    fig.add_trace(go.Bar(
        x=host_df['Year'],
        y=host_df['Team'],
        name=host
    ))
fig.update_layout(
    title='Times por Ano e Host',
    xaxis_title='Ano',
    yaxis_title='Times',
    barmode='group',
    legend_title='Host'
)
fig.show()

Interessante notar quedas abruptas em 2009 e 2020.

## Os times do país sede tem vantagem na competição?

A primeira pergunta que queremos responder é: em média, os times do país que está sediando a competição tem um desempenho melhor que os times de países visitantes?

In [ ]:
home = df[df['Host']==df['Country']][['Year', 'Rank']].groupby(['Year']).mean().rename(columns = { 'Rank':'Home' })
away = df[df['Host']!=df['Country']][['Year', 'Rank']].groupby(['Year']).mean().rename(columns = { 'Rank':'Away' })
pd.concat([home, away]).plot.line()

A comparação das equipes que estão jogando em casa e fora não demonstra um padrão tão claro, oscilando bastante de um ano para o outro. Por conta disso, suspeitamos que vantagens e desvantagens tivessem mais relação com a performance geral do país.

Logo, vamos investigar melhor essa diferença. Primeiro, qual a diferença do desempenho médio em edições em casa e fora pra cada um dos países que já foi sede? Como não houveram tantos países sedes até hoje na ICPC, conseguimos fazer um gráfico de barras com todos eles:

In [ ]:
# Calcula a média do rank de cada país em todas as edições
country_mean = df[['Country', 'Rank']].groupby('Country').mean().copy()

# Calcula a média do rank de cada país nas edições que sediou
country_host_mean = df[df['Host'] == df['Country']][['Country', 'Rank']].groupby('Country').mean().copy()

# Une os dois DataFrames e calcula a diferença
diff = country_host_mean.join(country_mean, on='Country', lsuffix='_host', rsuffix='_all')
diff['diff'] = diff['Rank_host'] - diff['Rank_all']

# Plota o gráfico da diferença
fig = go.Figure()
fig.add_trace(go.Bar(
    x=diff.index,
    y=diff['diff'],
    name='Sede - Geral',
    marker_color='blue'
))
fig.update_layout(
    title='Diferença de Desempenho dos Países Sede',
    xaxis_title='País',
    yaxis_title='Diferença de Rank (Sede - Geral)',
    xaxis_tickangle=-45,
)
fig.show()

Apesar da diferença favorecer as edições em casa para todos os países, observou-se pouca diferença (abaixo de 10 posições) na maior parte dos casos ao comparar os ranks médios gerais desses países ao longo dos anos com o das edições em que foram hosts. As exceções foram a República Tcheca, a Tailandia e a Suécia.

### Intervalo de Confiança desconsiderando outras variáveis (como ano/edição)

Comentário: acreditamos que o intervalo de confiança ofereça pouco no nosso contexto, pois possuímos os dados de toda a população de interesse, logo não é necessário aproximar a distribuição da população.

In [ ]:
# Calcula valor dos dados
diff_means = home['Home'].values.mean() - away['Away'].values.mean()

# Calcula IC da diferença média via boostrap
n_bootstraps = 10000
bootstrap_diffs = []
for _ in range(n_bootstraps):
  home_sample = np.random.choice(home['Home'].values, size=len(home['Home'].values), replace=True)
  away_sample = np.random.choice(away['Away'].values, size=len(away['Away'].values), replace=True)

  bootstrap_diff = np.mean(home_sample) - np.mean(away_sample)
  bootstrap_diffs.append(bootstrap_diff)

# Calcula intervalo de confiança com 5% de significância
confidence_interval = np.percentile(bootstrap_diffs, [2.5, 97.5])

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=bootstrap_diffs,
    nbinsx=30,
    name='Diferença das Médias',
    marker_color='blue',
    opacity=0.75,
))
fig.add_vline(x=confidence_interval[0], line=dict(color='red', dash='dash'), annotation_text='Limite Inferior')
fig.add_vline(x=confidence_interval[1], line=dict(color='red', dash='dash'), annotation_text='Limite Superior')
fig.add_vline(x=diff_means, line=dict(color='yellow', dash='dash'), annotation_text='Diferença Média Real')
fig.update_layout(
    title='Bootstrap da Distribuição da Diferença das Médias',
    xaxis_title='Diferença das Médias',
    yaxis_title='Frequência',
    legend_title='Legenda',
)
fig.show()

### Conclusão

Através das investigações acima, é possível concluir que os dados indicam que a localidade da competição não é um fator significativo no desempenho dos times.

## Como é o desempenho histórico de universidades brasileiras na competição?

A próxima pergunta é qual o desempenho histórico do Brasil na competição. Uma primeira métrica a ser observada é, naturalmente, o rank médio dos times brasileiros em todas as edições:

In [ ]:
df[df['Country'] == 'Brazil']['Rank'].mean()

63.08385431546581

A média dos ranks das equipes brasileiras seria o equivalente ao 63° lugar em 100.

Agora, vamos observar como a média brasileira varia em comparação com a média dos outros países:

In [ ]:
br = df[df['Country']=='Brazil'][['Year', 'Rank']].groupby(['Year']).mean().rename(columns = { 'Rank':'Brazil' })
others = df[df['Country']!='Brazil'][['Year', 'Rank']].groupby(['Year']).mean().rename(columns = { 'Rank':'Others' })
pd.concat([br, others]).plot.line()

É possível notar que o Brasil tende a ficar abaixo da média dos outros países, com uma participação particularmente ruim em 2004 e outra consideravelmente acima do esperado em 2006.

Vamos, pra fim de completude, calcular o intervalo de confiança dessa diferença:

In [ ]:
# Calcula valor dos dados
diff_means = br['Brazil'].values.mean() - others['Others'].values.mean()

# Calcula IC da diferença média via boostrap
n_bootstraps = 10000
bootstrap_diffs = []
for _ in range(n_bootstraps):
  br_sample = np.random.choice(br['Brazil'].values, size=len(br['Brazil'].values), replace=True)
  others_sample = np.random.choice(others['Others'].values, size=len(others['Others'].values), replace=True)

  bootstrap_diff = np.mean(br_sample) - np.mean(others_sample)
  bootstrap_diffs.append(bootstrap_diff)

# Calcula intervalo de confiança com 5% de significância
confidence_interval = np.percentile(bootstrap_diffs, [2.5, 97.5])

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=bootstrap_diffs,
    nbinsx=30,
    name='Diferença das Médias',
    marker_color='blue',
    opacity=0.75
))
fig.add_vline(x=confidence_interval[0], line=dict(color='red', dash='dash'), annotation_text='Limite Inferior')
fig.add_vline(x=confidence_interval[1], line=dict(color='red', dash='dash'), annotation_text='Limite Superior')
fig.add_vline(x=diff_means, line=dict(color='yellow', dash='dash'), annotation_text='Diferença Média Real')
fig.update_layout(
    title='Bootstrap da Distribuição da Diferença das Médias',
    xaxis_title='Diferença das Médias',
    yaxis_title='Frequência',
    legend_title='Legenda',
    template='plotly_dark'
)
fig.show()

### Conclusão

É possível concluir que o desempenho médio brasileiro é um pouco abaixo da média global. Além disso, é possível ver que o país tem um desempenho bem inconsistente ao longo das competições, o que reflete em um IC que favorece a média brasileira.

## Qual o desempenho médio de cada país?

Agora, vamos investigar o desempenho médio dos países como um todo e tentar predizer manualmente quais seriam os países mais fortes na competição.

Começamos com um scatterplot do desempenho médio:

In [ ]:
country_means = df.groupby('Country')['Rank'].mean().sort_values(ascending=False)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=country_means.values,
    y=country_means.index,
    mode='markers',
    marker=dict(color='blue', size=10),
))
fig.update_layout(
    title='Dispersão do Desempenho Médio de Cada País',
    xaxis_title='Média do Ranking',
    yaxis_title='País',
    yaxis=dict(tickmode='linear'),
    height=1200,
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=country_means.index,
    z=country_means.values,
    locationmode='country names',
    colorscale='Blues_r',
    colorbar_title='Média do Ranking',
))

fig.update_layout(
    title_text='Mapa Mundial do Desempenho Médio por País',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
)

fig.show()

Comparando o gráfico acima, que mostra o desempenho médio dos países, com o gráfico abaixo, que mostra o desempenho médio do melhor time de cada país, é possível notar uma certa inversão no topo. Países como Estados Unidos e Rússia tem médias relativamente ruins, mas melhores desempenhos muito altos. Isso se deve ao fato desses países terem um número de participantes grande comparado aos outros, o que é ilustrado logo abaixo:

In [ ]:
df_2021 = df[df['Year'] == 2021]

teams_per_country = df_2021.groupby('Country')['Team'].nunique()
top_countries = teams_per_country.nlargest(2)
average_teams_per_country = teams_per_country.mean()

countries = top_countries.index.tolist() + ['Média Global']
team_counts = top_countries.tolist() + [average_teams_per_country]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=countries,
    y=team_counts,
    name='Número de Times',
    marker_color=['blue'] * 2 + ['red']
))
fig.add_trace(go.Scatter(
    x=countries,
    y=[average_teams_per_country] * len(countries),
    mode='lines+markers',
    name='Média Global',
    line=dict(color='royalblue', width=2),
    marker=dict(size=10)
))
fig.update_layout(
    title='Países com Mais Times em 2021',
    xaxis_title='País',
    yaxis_title='Número de Times',
)
fig.show()

In [ ]:
best_teams = df.groupby(['Country', 'Year'])['Rank'].min().reset_index()
country_best_means = best_teams.groupby('Country')['Rank'].mean().sort_values(ascending=False)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=country_best_means.values,
    y=country_best_means.index,
    mode='markers',
    marker=dict(color='blue', size=10),
))
fig.update_layout(
    title='Dispersão do Desempenho Médio do Melhor Time de Cada País',
    xaxis_title='Média do Ranking do Melhor Time',
    yaxis_title='País',
    yaxis=dict(tickmode='linear'),
    height=1200,
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=country_best_means.index,
    z=country_best_means.values,
    locationmode='country names',
    colorscale='Blues_r',
    colorbar_title='Média do Ranking do Melhor Time',
))

fig.update_layout(
    title_text='Mapa Mundial do Desempenho Médio do Melhor Time por País',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
)

fig.show()

Para investigar melhor essa diferença do melhor desempenho e o desempenho médio, criamos um gráfico da média da diferença entre o melhor e pior time de cada país:

In [ ]:
# Calcula a diferença entre o melhor e o pior time de cada país por ano
best_teams = df.groupby(['Country', 'Year'])['Rank'].min().reset_index().copy()
worst_teams = df.groupby(['Country', 'Year'])['Rank'].max().reset_index().copy()
diff_teams = best_teams.merge(worst_teams, on=['Country', 'Year'], suffixes=('_best', '_worst'))
diff_teams['diff'] = diff_teams['Rank_worst'] - diff_teams['Rank_best']

# Calcula a média anual da diferença
mean_diff = diff_teams.groupby('Country')['diff'].mean().sort_values()

# filtra apenas os países com diferença maior que zero
mean_diff = mean_diff[mean_diff > 0]

# Cria o scatterplot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mean_diff.values,
    y=mean_diff.index,
    mode='markers',
    marker=dict(color='blue', size=10),
))
fig.update_layout(
    title='Dispersão da Média Anual da Diferença entre o Melhor e o Pior Time de Cada País',
    xaxis_title='Média Anual da Diferença',
    yaxis_title='País',
    yaxis=dict(tickmode='linear'),
    height=900,
)
fig.show()

Para ter uma visualização melhor da distribuição desses valores, fizemos o scatterplot abaixo. É possível notar que essas correlações são enviezadas pelo fato de que a absoluta maioria dos países tem uma participação muito pequena e alguns poucos países tem muitos participantes. É possível ver quais são os outliers passando o mouse: China, Russia e Estados Unidos.

In [ ]:
average_num_participants = df.groupby(['Country', 'Year'])['Team'].nunique().groupby('Country').mean()
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=average_num_participants.values,
    y=country_best_means.values,
    mode='markers',
    text=average_num_participants.index,
    marker=dict(size=10, color='blue')
))

# linha vertical na mediana:
median_value = average_num_participants.median()
fig.add_vline(x=median_value, line=dict(color='red', width=2, dash='dash'), name=f'Mediana = {median_value}')

fig.update_layout(
    title='Número Médio de Participantes vs Média do Melhor Time',
    xaxis_title="Número Médio de Participantes",
    yaxis_title="Média do Melhor Time",
)
fig.show()

Por fim, vamos plotar a matriz de correlação de todas essas métricas:

In [ ]:
data = pd.concat([country_means, country_best_means, average_num_participants, mean_diff], axis=1)
corr_matrix = data.corr()

tick_labels = ['Média do Ranking', 'Média do Ranking do Melhor Time', 'Número Médio de Participantes', 'Média Anual da Diferença']
fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='solar',
    labels=dict(x="", y="", color="Correlação"),
    x=tick_labels,
    y=tick_labels
)
fig.update_layout(title='Heatmap da Correlação')
fig.show()

In [ ]:
lotsa_teams = average_num_participants[average_num_participants > 3].sort_values()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=lotsa_teams.values,
    y=lotsa_teams.index,
    mode='markers',
    marker=dict(color='blue', size=10),
))
fig.update_layout(
    title='Países com Mais de 3 Participações em Média por Edição',
    xaxis_title='Média de Participantes/Edição',
    yaxis_title='País',
    yaxis=dict(tickmode='linear')
)
fig.show()

### Conclusão

China, Rússia e Estados Unidos parecem ser os países mais competitivos, considerando tanto o desempenho dos melhores times quanto o número de times classificados.

# Predição e Classificação

Agora, partimos para a utilização de modelos de aprendizado de máquina para tentar predizer métricas e realizar classificações

## Países Competitivos vs Não Competitivos

Vamos comparar a nossa análise manual prévia com o resultado de um modelo de classificação:

Primeiro, tentamos classificar os países que tem desempenho médio global melhor que a mediana:

In [ ]:
average_participations = df.groupby('Country')['Team'].count().median().copy()
dfa = df.groupby('Country').filter(lambda x: len(x) >= average_participations).copy()

# Define um limiar para separar países competitivos de não competitivos
threshold = dfa['Rank'].median()

# Cria uma nova coluna 'Competitive' com base no limiar
dfa['Competitive'] = dfa['Rank'] <= threshold

# Prepara os dados
features = dfa[['Country']].copy()
target = dfa['Competitive'].copy()

# Converte as features categóricas para numéricas usando one-hot encoding
features = pd.get_dummies(features, columns=['Country'])

# Divide os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Treina um modelo RandomForestClassifier
model = RandomForestClassifier(random_state=57)
model.fit(X_train, y_train)

# Faz previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avalia o modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo no conjunto de teste:", accuracy)


Acurácia do modelo no conjunto de teste: 0.7096018735362998


In [ ]:
def print_feature_importance(model, features):
  importances = model.feature_importances_

  feature_names = [name.replace('Country_', '') for name in features.columns]
  importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
  importance_df = importance_df.sort_values(by='Importance', ascending=False)

  fig = go.Figure(go.Bar(
      x=importance_df['Importance'],
      y=importance_df['Feature'],
      orientation='h',
      marker=dict(color='blue')
  ))
  fig.update_layout(
      title='Importância de Features - Contribuição para Competitividade',
      xaxis_title='Importância',
      yaxis_title='País',
      yaxis=dict(autorange="reversed"),
      height=900,
  )
  fig.show()
print_feature_importance(model, features)

In [ ]:
# Obtém os nomes das colunas (países)
country_columns = [col for col in features.columns if col.startswith('Country_')]

# Obtém os índices dos países preditos como competitivos
competitive_indices = model.predict(features) == True

# Mapeia os índices para os nomes dos países
predicted_competitive_countries = [country_columns[i].replace('Country_', '') for i in range(len(country_columns)) if competitive_indices[i]]

competitive = dfa[dfa['Country'].isin(predicted_competitive_countries)].groupby('Country')['Rank'].mean().sort_values()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=competitive.values,
    y=competitive.index,
    mode='markers',
    marker=dict(color='blue', size=10),
))
fig.update_layout(
    title='Países Preditos como Competitivos',
    xaxis_title='Média do Rank dos Times',
    yaxis_title='País',
    yaxis=dict(tickmode='linear', autorange='reversed'),
)
fig.show()

Segue abaixo a matriz de confusão do modelo:

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred, normalize= 'true')

def print_confusion(cm):
  fig = px.imshow(
      cm,
      text_auto=True,
      color_continuous_scale='solar',
      labels=dict(x="Predito", y="Verdadeiro", color="Confusão"),
      x=['Não Competitivo', 'Competitivo'],
      y=['Não Competitivo', 'Competitivo']
  )
  fig.update_layout(title='Matriz de Confusão')
  fig.show()
print_confusion(cm)

Agora, vamos utilizar o desempenho do melhor time, para o critério ser o mesmo da nossa última análise manual:

In [ ]:
dfa = df.copy()

# Filtra os desempenhos para conter apenas o melhor time em cada ano
dfa = dfa.groupby(['Country', 'Year'])['Rank'].min().reset_index().copy()

# Define um limiar para separar países competitivos de não competitivos
threshold = dfa['Rank'].median()

# Cria uma nova coluna 'Competitive' com base no limiar
dfa['Competitive'] = dfa['Rank'] <= threshold

# Prepara os dados
features = dfa[['Country']].copy()
target = dfa['Competitive'].copy()

# Converte as features categóricas para numéricas usando one-hot encoding
features = pd.get_dummies(features, columns=['Country'])

# Divide os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Treina um modelo RandomForestClassifier
model = RandomForestClassifier(random_state=69)
model.fit(X_train, y_train)

# Faz previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avalia o modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo no conjunto de teste:", accuracy)

Acurácia do modelo no conjunto de teste: 0.7125


In [ ]:
print_feature_importance(model, features)

In [ ]:
# Obtém os nomes das colunas (países)
country_columns = [col for col in features.columns if col.startswith('Country_')]

# Obtém os índices dos países preditos como competitivos
competitive_indices = model.predict(features) == True

# Mapeia os índices para os nomes dos países
predicted_competitive_countries = [country_columns[i].replace('Country_', '') for i in range(len(country_columns)) if competitive_indices[i]]

competitive = dfa[dfa['Country'].isin(predicted_competitive_countries)].groupby('Country')['Rank'].min().sort_values()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=competitive.values,
    y=competitive.index,
    mode='markers',
    marker=dict(color='blue', size=10),
))
fig.update_layout(
    title='Países Preditos como Competitivos',
    xaxis_title='Rank do Melhor Time',
    yaxis_title='País',
    yaxis=dict(tickmode='linear', autorange='reversed'),
)
fig.show()

Segue abaixo a matriz de confusão do modelo:

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred, normalize= 'true')
print_confusion(cm)

Podemos observar que, apesar da acurácia ainda ser relativamente baixa, a lista dos países competivos agora contém tanto a China quanto os Estados Unidos, os dois países com mais times e, portanto, os mais afetados pela mudança da média dos times para o melhor time.

## Predição do Rank de um Time

Agora, vamos tentar predizer o Rank de um time. As variáveis que podem ser explicativas são:



*   País
*   Sede da edição
*   "Score Percentage": Essa feature indica a porcentagem de questões da prova que o time resolveu, o que significa que essa métrica já é normalizada.
*   "Penalty": Os times recebem uma certa quantidade de penalidade baseada no tempo que demoraram para resolver cada questão + um adicional a cada submissão que falha.



Vamos então, calcular a matriz de correlação das features númericas:

In [ ]:
features_selected = ['Score Percentage', 'Penalty']

fig = px.imshow(
    df[features_selected].corr(),
    text_auto=True,
    color_continuous_scale='solar',
    labels=dict(x="", y="", color="Correlação"),
    x=features_selected,
    y=features_selected
)
fig.update_layout(title='Correlação das Features Numéricas')
fig.show()

A correlação existe, o que é de se esperar porque um time sofre penalidades de acordo com o tempo que demoram para resolver um problema corretamente, ou a quantidade de tentativas erradas caso não consigam. Assim, um time com mais problemas recebe maior penalidade na média.

Por isso, vamos investigar também se um modelo com apenas uma delas consegue ser melhor que um que use as duas.

Começamos com um modelo que usa todas as features:

In [ ]:
models = []

In [ ]:
features_selected = ['Country', 'Host', 'Score Percentage', 'Penalty']
dfa = df.dropna(subset=features_selected)
features = dfa[features_selected].copy()
target = dfa['Rank'].copy()

features = pd.get_dummies(features, columns=['Country', 'Host'])

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
models.append(model)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
lr_mse_all_feats = mse

model = RandomForestRegressor(random_state=32)
model.fit(X_train, y_train)
models.append(model)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rf_mse_all_feats = mse

mse_values = [lr_mse_all_feats, rf_mse_all_feats]
models_names = ['Rregressão Linear', 'Random Forest']
fig = go.Figure(go.Bar(
    x=models_names,
    y=mse_values,
    marker=dict(color=['red', 'blue']),
    text=[f'{mse:.2f}' for mse in mse_values],  # Display MSE values on bars
    textposition='auto'
))
fig.update_layout(
    title='Erro Médio no Conjunto de Teste (Modelos com Todas as Features)',
    xaxis_title='Modelo',
    yaxis_title='Mean Squared Error'
)
fig.show()

Agora, vamos ver se um modelo com apenas as variáveis númericas consegue um desempenho melhor, o que seria surpreendente:

In [ ]:
features_selected = ['Score Percentage', 'Penalty']
dfa = df.dropna(subset=features_selected)
features = dfa[features_selected].copy()
target = dfa['Rank'].copy()

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
models.append(model)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
lr_mse_num_feats = mse

model = RandomForestRegressor(random_state=12)
model.fit(X_train, y_train)
models.append(model)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rf_mse_num_feats = mse

mse_values = [lr_mse_num_feats, rf_mse_num_feats]
models_names = ['Rregressão Linear', 'Random Forest']
fig = go.Figure(go.Bar(
    x=models_names,
    y=mse_values,
    marker=dict(color=['red', 'blue']),
    text=[f'{mse:.2f}' for mse in mse_values],  # Display MSE values on bars
    textposition='auto'
))
fig.update_layout(
    title='Erro Médio no Conjunto de Teste (Modelos com Features Numéricas)',
    xaxis_title='Modelo',
    yaxis_title='Mean Squared Error'
)
fig.show()

Como imaginavamos, as features categóricas ajudam. Finalmente, vamos testar usar apenas uma das features numéricas:

In [ ]:
features_selected = ['Country', 'Host', 'Score Percentage']
dfa = df.dropna(subset=features_selected)
features = dfa[features_selected].copy()
target = dfa['Rank'].copy()

features = pd.get_dummies(features, columns=['Country', 'Host'])

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
models.append(model)


y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
lr_mse_score_percentage = mse

model = RandomForestRegressor(random_state=32)
model.fit(X_train, y_train)
models.append(model)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rf_mse_score_percentage = mse

mse_values = [lr_mse_score_percentage, rf_mse_score_percentage]
models_names = ['Rregressão Linear', 'Random Forest']
fig = go.Figure(go.Bar(
    x=models_names,
    y=mse_values,
    marker=dict(color=['red', 'blue']),
    text=[f'{mse:.2f}' for mse in mse_values],  # Display MSE values on bars
    textposition='auto'
))
fig.update_layout(
    title='Erro Médio no Conjunto de Teste (Modelos com Apenas Score Percentage)',
    xaxis_title='Modelo',
    yaxis_title='Mean Squared Error'
)
fig.show()

Segue abaixo um resumo do erro quadrático médio no conjunto de teste para todos os modelos:

In [ ]:
data = [
    ["Regressão Linear", "Todas", lr_mse_all_feats],
    ["Random Forest", "Todas", rf_mse_all_feats],
    ["Regressão Linear", "Numéricas", lr_mse_num_feats],
    ["Random Forest", "Numéricas", rf_mse_num_feats],
    ["Regressão Linear", "Score", lr_mse_score_percentage],
    ["Random Forest", "Score", rf_mse_score_percentage]
]
data = pd.DataFrame(data, columns=["Modelo", "Features", "MSE"])

fig = px.imshow(
    data.pivot(index="Modelo", columns="Features", values="MSE"),
    text_auto='.1e',
    color_continuous_scale='solar_r',
    x=['Numéricas', 'Score', 'Todas'],
    y=['Random Forest', 'Regressão Linear']
)
fig.update_layout(title='Desempenho dos Modelos')
fig.show()

Aparentemente, o melhor modelo se trata do RandomForestClassifier utilizando de todas as features pré-selecionadas. Vamos ver qual a posição que ele prediz pra um dado fictício, de um time brasileiro fazendo uma prova nos Estados Unidos, com diferentes scores. Podemos mudar os países e a penalidade para fins de curiosidade mudando os widgets.

In [ ]:
#@title Resultado Interativo
from ipywidgets import interact, widgets

model = models[1]

def plot_predictions(country, host, penalty):
    team_data = {
        'Country': country,
        'Host': host,
        'Score Percentage': 0.00,
        'Penalty': penalty
    }

    team_df = pd.DataFrame([team_data])

    features_selected = ['Country', 'Host', 'Score Percentage', 'Penalty']
    dfa = df.dropna(subset=features_selected)
    features = dfa[features_selected].copy()
    features = pd.get_dummies(features, columns=['Country', 'Host'])

    team_df_encoded = pd.get_dummies(team_df, columns=['Country', 'Host'])
    team_df_encoded = team_df_encoded.reindex(columns=features.columns, fill_value=0)

    percentages = np.arange(0.1, 1.1, 0.1)
    predictions = []
    for pct in percentages:
        team_df_encoded['Score Percentage'] = pct
        prediction = model.predict(team_df_encoded)[0]
        predictions.append(prediction)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=percentages * 100,
        y=predictions,
        mode='lines+markers',
        name='Rank Predito',
        marker=dict(size=8)
    ))
    fig.update_layout(
        title=f'Rank Predito para um Time do {country} em {host}',
        xaxis_title='Porcentagem de Score (%)',
        yaxis_title='Rank Predito (%)',
        yaxis=dict(autorange="reversed"),
    )
    fig.show()

countries = df['Country'].unique()
hosts = df['Host'].unique()
penalty_range = (0, 5000, 100)

plot = interact(plot_predictions,
    country=widgets.Dropdown(options=countries, value='Brazil', description='Country:'),
    host=widgets.Dropdown(options=hosts, value='United States', description='Host:'),
    penalty=widgets.IntSlider(min=penalty_range[0], max=penalty_range[1], step=penalty_range[2], value=1000, description='Penalty:')
  )


interactive(children=(Dropdown(description='Country:', index=16, options=('Canada', 'Germany', 'Russia', 'Roma…

 É possível ver que a final mundial se trata de uma prova difícil, visto que um time que fez apenas metade da prova é predito como ficando por volta do 30° lugar (em 100), e que o modelo não pensa que nenhum time seja capaz de ser campeão mesmo fechando a prova!

### Conclusão

 É possível predizer de forma razoável o posicionamento final de um time baseado em suas métricas de performance e informações como o país do time e o país sede. Porém, fica evidente que o resultado é longe de perfeito, o que é evidenciado pelo erro quadrado no conjunto de teste. Logo, outros fatores também são significativos no resultado final dos times.